In [27]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import metrics
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense




In [28]:
ACTIVATION_1 = 'relu'
ACTIVATION_2 = 'sigmoid'
ACTIVATION_3 = 'sigmoid'

EPOCHS = 10

LEARNING_RATE=0.001

METRICS=   ['mae', 'mse']

NEURON_LAYER_1 =  32
NEURON_LAYER_2 =  32
NEURON_LAYER_3 = 32

PERCENT_TRAINNING = 0.60 

In [29]:
CSV_COLUMNS = ["stock_item_id","sales_price","sales_quantity",
               "diference_price","sales_tax_rate","sales_tax_amount",
               "last_cost_Price","quantity_per_Outer","tax_rate",
               "recommended_retail_price"]

raw_dataset = pd.read_csv('exported_data_1.csv', names=CSV_COLUMNS ,
                      na_values = "?", comment='\t',
                      sep=";", skipinitialspace=True,encoding = 'ISO-8859-1')

dataset = raw_dataset.copy()


dataset = dataset.dropna()
dataset.describe()

,stock_item_id,sales_price,sales_quantity,diference_price,sales_tax_rate,sales_tax_amount,last_cost_Price,quantity_per_Outer,tax_rate,recommended_retail_price
count,228265.000000,228265.000000,228265.000000,228265.000000,228265.000000,228265.000000,228265.000000,228265.000000,228265.000000,228265.000000
mean,110.181285,45.591689,39.211566,22.755450,14.977307,112.948101,23.289794,7.204140,14.977307,68.347139
std,63.729035,139.862055,54.558829,69.473497,0.336081,217.512539,83.156475,8.197167,0.336081,209.150176
min,1.000000,0.660000,1.000000,0.330000,10.000000,0.380000,0.360000,1.000000,10.000000,0.990000
25%,54.000000,13.000000,5.000000,6.440000,15.000000,14.400000,4.500000,1.000000,15.000000,19.440000
50%,111.000000,18.000000,10.000000,8.910000,15.000000,34.500000,8.000000,1.000000,15.000000,26.910000
75%,165.000000,32.000000,60.000000,15.840000,15.000000,129.600000,15.500000,12.000000,15.000000,47.840000
max,227.000000,1899.000000,360.000000,940.010000,15.000000,2848.500000,1140.000000,36.000000,15.000000,2839.010000


In [30]:
train_dataset = dataset.sample(frac=PERCENT_TRAINNING,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
test_dataset_copy = test_dataset

train_stats = train_dataset.describe()
train_stats.pop("sales_price")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
stock_item_id,136959.0,110.019962,63.734388,1.00,54.00,111.00,165.00,227.00
sales_quantity,136959.0,39.142911,54.443931,1.00,5.00,10.00,60.00,360.00
diference_price,136959.0,22.524254,68.386613,0.33,6.44,8.91,15.84,940.01
sales_tax_rate,136959.0,14.977548,0.334300,10.00,15.00,15.00,15.00,15.00
sales_tax_amount,136959.0,112.482198,216.523676,0.38,14.40,34.50,129.60,2848.50
last_cost_Price,136959.0,23.038628,81.879160,0.36,4.50,8.00,15.50,1140.00
quantity_per_Outer,136959.0,7.198052,8.191716,1.00,1.00,1.00,12.00,36.00
tax_rate,136959.0,14.977548,0.334300,10.00,15.00,15.00,15.00,15.00
recommended_retail_price,136959.0,67.682379,205.964003,0.99,19.44,26.91,47.84,2839.01


In [31]:
train_labels = train_dataset.pop('sales_price')
test_labels = test_dataset.pop('sales_price')
stockItemIdArray = test_dataset.get('stock_item_id')
train_dataset.head(10)

,stock_item_id,sales_quantity,diference_price,sales_tax_rate,sales_tax_amount,last_cost_Price,quantity_per_Outer,tax_rate,recommended_retail_price
118448,16,7,6.44,15.0,13.65,4.50,1,15.0,19.44
22212,82,12,8.91,15.0,32.40,7.50,12,15.0,26.91
106489,188,200,0.33,15.0,19.80,0.36,25,15.0,0.99
7764,146,108,8.91,15.0,291.60,15.00,12,15.0,26.91
222629,154,60,9.90,15.0,180.00,10.00,10,15.0,29.90
197688,190,200,1.73,15.0,105.00,1.80,20,15.0,5.23
112111,208,120,1.34,15.0,48.60,1.70,12,15.0,4.04
203105,80,84,8.91,15.0,226.80,7.00,12,15.0,26.91
78094,105,1,17.33,15.0,5.25,17.00,1,15.0,52.33
142530,6,10,15.84,15.0,48.00,12.00,1,15.0,47.84


In [32]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [33]:
def build_model():
  
  layersArray= [
    layers.Dense(NEURON_LAYER_1, activation=ACTIVATION_1, 
                 input_shape=[len(train_dataset.keys())]),
    layers.Dense(NEURON_LAYER_2, activation=ACTIVATION_2),
  ]

  if NEURON_LAYER_3>0:
    layersArray.append(layers.Dense(NEURON_LAYER_3, activation=ACTIVATION_3)) 

  layersArray.append(layers.Dense(1))

  model = keras.Sequential(layersArray)

  OPTIMIZER= tf.keras.optimizers.Adam(LEARNING_RATE) 
  #OPTIMIZER= tf.keras.optimizers.RMSprop(LEARNING_RATE)
  
  model.compile(loss=METRICS[1],optimizer=OPTIMIZER,metrics=METRICS)
  
  return model


In [34]:
model = build_model()
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                320       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 2,465
Trainable params: 2,465
Non-trainable params: 0
_________________________________________________________________


In [35]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

array([[-0.44272247],
       [-0.43886644],
       [-0.5008209 ],
       [-0.47602993],
       [-0.44415188],
       [-0.49959004],
       [-0.44668415],
       [-0.47559094],
       [-0.42990252],
       [-0.44923428]], dtype=float32)

In [36]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('Inicio')
    print('-',epoch, end='')

In [ ]:
history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,callbacks=[PrintDot()])


Inicio
- 0- 1- 2- 3- 4- 5

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  if 'mae' in hist:
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [salesPrice]')
    plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
    plt.ylim([0,5])
    plt.legend()
  
  if 'mse' in hist:
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$sales_price^2$]')
    plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
    plt.ylim([0,20])
    plt.legend()


  plt.show()


  
plot_history(history)


In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [salesPrice]')
plt.ylabel('Predictions [salesPrice]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 2000], [-100, 2000])



In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [salesPrice]")
_ = plt.ylabel("Count")


In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} sales_price".format(mae))






In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [sales_price]")
_ = plt.ylabel("Count")


In [ ]:
for stockItemId,originalValue,prediction in zip(stockItemIdArray,test_labels,test_predictions):
  print("stock_item_id:",stockItemId,"Original Value:",originalValue,"prediction Value:",prediction)